In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [5]:
# Load the dataset
dataset = pd.read_csv('GOOGL.csv', index_col='Date', parse_dates=['Date'])

In [4]:
from google.colab import files
uploaded = files.upload()

Saving GOOGL.csv to GOOGL.csv


In [6]:
# Split the dataset into training and testing sets
train_set = dataset[:'2018'].iloc[:, 1:2].values
test_set = dataset['2019':].iloc[:, 1:2].values


In [7]:
# Normalize the dataset
sc = MinMaxScaler(feature_range=(0, 1))
train_set_scaled = sc.fit_transform(train_set)


In [8]:
# Create a function to generate a sliding window of data
def create_dataset(dataset, look_back=60):
    X, Y = [], []
    for i in range(look_back, len(dataset)):
        X.append(dataset[i-look_back:i, 0])
        Y.append(dataset[i, 0])
    X, Y = np.array(X), np.array(Y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, Y


In [9]:
# Generate the training and testing datasets
look_back = 60
X_train, Y_train = create_dataset(train_set_scaled, look_back)
X_test, Y_test = create_dataset(test_set, look_back)


In [10]:
# Build the RNN model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')


In [11]:
# Train the RNN model
model.fit(X_train, Y_train, epochs=100, batch_size=32)


Epoch 1/100
112/112 [==============================] - 14s 79ms/step - loss: 0.0102
Epoch 2/100
112/112 [==============================] - 8s 71ms/step - loss: 2.3266e-04
Epoch 3/100
112/112 [==============================] - 8s 69ms/step - loss: 2.2142e-04
Epoch 4/100
112/112 [==============================] - 9s 78ms/step - loss: 2.2273e-04
Epoch 5/100
112/112 [==============================] - 7s 66ms/step - loss: 2.1726e-04
Epoch 6/100
112/112 [==============================] - 9s 78ms/step - loss: 2.1185e-04
Epoch 7/100
112/112 [==============================] - 9s 84ms/step - loss: 2.1656e-04
Epoch 8/100
112/112 [==============================] - 7s 67ms/step - loss: 2.3223e-04
Epoch 9/100
112/112 [==============================] - 9s 81ms/step - loss: 1.9187e-04
Epoch 10/100
112/112 [==============================] - 9s 77ms/step - loss: 1.9137e-04
Epoch 11/100
112/112 [==============================] - 7s 66ms/step - loss: 1.8098e-04
Epoch 12/100
112/112 [======================

In [12]:
# Evaluate the model on the testing set
mse = model.evaluate(X_test, Y_test)
rmse = np.sqrt(mse)


24/24 [==============================] - 1s 19ms/step - loss: 3842086.7500


In [13]:
# Make predictions on future Google stock prices
last_60_days = train_set[-60:]
last_60_days_scaled = sc.transform(last_60_days)
X_pred = np.array([last_60_days_scaled])
X_pred = np.reshape(X_pred, (X_pred.shape[0], X_pred.shape[1], 1))
pred_price = model.predict(X_pred)
pred_price = sc.inverse_transform(pred_price)
print(pred_price)


1/1 [==============================] - 1s 960ms/step
[[1058.1979]]
